In [ ]:
# How to run: Should be pasted in solution.py and used in main.ipynb
def find_optimal_pruning(model, train_loader, test_loader, device, prune_ratio_dict: Dict, steps):
  # This functions helps to find the optimal pruning ratio to futher reduce energy consuption while maintaining above 85% accuracy.
    ## Initial Prune ratio
    init_prune_ratio = 0.3
    testAccuracy = 100

    while testAccuracy > 85 :
        init_prune_ratio += steps

        # Update prune ratio
        for key in prune_ratio_dict:
            prune_ratio_dict[key] = init_prune_ratio

        print(prune_ratio_dict)
        # Prune
        sparsed_model = model_to_spars(model, prune_ratio_dict)
        print("Prune ratio: ", init_prune_ratio)
        print("Pruned Model size:", model_size(sparsed_model))

        # Fine-tune
        print("Fine-tuning.....")
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(sparsed_model.parameters(), 1e-3, momentum=0.9, weight_decay=0.0005, nesterov=True)
        scheduler = None
        train_accuracy, test_accuracy = fit(sparsed_model, 5, train_loader, test_loader, criterion, optimizer, scheduler, device)
        testAccuracy = int(max(test_accuracy))

    # Return optimal prune ratio
    return init_prune_ratio - steps